In [89]:
import numpy as np
import pandas as pd
import random
import cv2
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
# from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras import backend as k
import os

In [90]:
def load(paths, verbose=-1):
    data = list()
    labels = list()
    # Loop over input images
    for (i,imgpath) in enumerate(paths):
        #  load image and extract label
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        image = np.array(im_gray).flatten()
        label = imgpath.split(os.path.sep)[-2]
        #  scale image
        data.append(image/255)
        labels.append(label)
        #  show update
        if verbose > 0 and i > 0 and (i+1) % verbose ==  0:
            print("[INFO] processed {}/{}".format(i + 1, len(paths)))
        #  return tuple mof data and label
    return data,labels

In [91]:
#  declare path
img_path = '/home/nextg3/Documents/FederatedLearning/FL_final/MNIST_training'
# Get path list
image_paths = list(paths.list_images(img_path))
#  apply function
image_list, label_list = load(image_paths, verbose = 10000)
#  Binarize the labels
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)
# split train test
X_train,X_test, y_train, y_test = train_test_split(image_list,
                                                   label_list,
                                                   test_size=0.1,
                                                   random_state=37)

[INFO] processed 10000/60000
[INFO] processed 20000/60000
[INFO] processed 30000/60000
[INFO] processed 40000/60000
[INFO] processed 50000/60000
[INFO] processed 60000/60000


In [92]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    '''returns a dictionary with keys as client names and values as data shards - (image, label)
        args:
    
                '''
    #  create list of client names
    client_names = ['{}_{}'.format(initial,i) for i in range(num_clients)]
    #  randomize data
    data = list(zip(image_list,label_list))
    random.shuffle(data)
    #  place data at each client
    size = len(data)//num_clients
    shards = [data[i:i+size] for i in range(0, size*num_clients, size)]

    # no of shrads should equa no of clients
    assert(len(shards)==len(client_names))

    return {client_names[i]:shards[i] for i in range(len(client_names))}

In [93]:
#  create clients
clients = create_clients(X_train,y_train,num_clients=10, initial='client')

In [94]:
def batch_data(data_shard, bs=32):
    '''client data shard -> tfds object
        args:
            shard
            batch size
        return
            tfds object'''
    #  sep data and label
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data),list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [95]:
# Process and batch data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#  Processed and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(len(y_test))

In [96]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [97]:
lr = 0.01
coms_round = 100
loss = 'categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate = lr,
                momentum = 0.9
                )

In [98]:
def weight_scaling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    # get bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    # Calculater the total data points across the clients
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get total no of data points held by the client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''Function for scaling a model's weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar*weight[i])
    return weight_final


In [99]:
def  sum_scaled_weights(scaled_weight_list):
    '''Returns the sum of the listed scaled weights.
    scaled average of the weights'''
    avg_grad = list()
    #  get the average grad over all client gradients
    for grad_list_tupel in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tupel, axis=0)
        avg_grad.append(layer_mean)
    return avg_grad


def test_model(X_test, Y_test, model, coms_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    # logits=model.predict(X_test, batch_size=100)
    logits=model.predict(X_test)
    loss = cce(y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis  = 1), tf.argmax(Y_test, axis = 1))
    print('coom_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss


In [102]:
# Initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784,10)

comms_round = 100

# start global training loop
for comm_round in range(comms_round):
    #  get the global model's weights - initial weights for all local models
    global_weights = global_model.get_weights()

    # List to collect local model weights after scaling
    scaled_local_weight_list = list()

    # Randomize client data using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # Loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784,10)
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)
        # set weights of the global model as weights of the local modelk
        local_model.set_weights(global_weights)

        # Fit local model with client data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        # scale the model weights and add to the list
        scaling_factor = weight_scaling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(),scaling_factor)
        scaled_local_weightsmlp_local = SimpleMLP()
        local_model = smlp_local.build(784,10)
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)
        # set weights of the global model as weights of the local modelk
        local_model.set_weights(global_weights)

        # Fit local model with client data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        # scale the model weights and add to the list
        scaling_factor = weight_scaling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(),scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # Clear session to clean memory
        k.clear_session()

    # update global model
    global_model.set_weights(average_weights)

    # Test globalmodel
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)


188/188 [==============================] - 0s 625us/step
coom_round: 0 | global_acc: 90.000% | global_loss: 1.6354068517684937
188/188 [==============================] - 0s 628us/step
coom_round: 1 | global_acc: 92.217% | global_loss: 1.5910351276397705
188/188 [==============================] - 0s 609us/step
coom_round: 2 | global_acc: 93.317% | global_loss: 1.5720058679580688
188/188 [==============================] - 0s 627us/step
coom_round: 3 | global_acc: 94.233% | global_loss: 1.5576733350753784
188/188 [==============================] - 0s 607us/step
coom_round: 4 | global_acc: 94.733% | global_loss: 1.5493097305297852
188/188 [==============================] - 0s 624us/step
coom_round: 5 | global_acc: 95.183% | global_loss: 1.5399550199508667
188/188 [==============================] - 0s 617us/step
coom_round: 6 | global_acc: 95.367% | global_loss: 1.5345598459243774
188/188 [==============================] - 0s 663us/step
coom_round: 7 | global_acc: 95.767% | global_loss: 1.5

In [114]:
str(average_weights) 

'[<tf.Tensor: shape=(784, 200), dtype=float32, numpy=\narray([[ 0.03571758, -0.02571049, -0.02710956, ...,  0.05763042,\n         0.05246304,  0.06008835],\n       [ 0.0049664 , -0.07706694, -0.00089854, ...,  0.07466882,\n         0.06001882,  0.06352755],\n       [-0.02584577,  0.07723933,  0.0243814 , ..., -0.05433658,\n         0.00056902,  0.06095746],\n       ...,\n       [-0.00387093, -0.0413771 ,  0.05094896, ..., -0.03800261,\n        -0.02351406, -0.05670906],\n       [ 0.02975903, -0.00170064,  0.00195459, ..., -0.05663487,\n        -0.0453493 , -0.04946703],\n       [ 0.00289438,  0.06778017, -0.03451001, ...,  0.04838387,\n         0.07487383,  0.06311972]], dtype=float32)>, <tf.Tensor: shape=(200,), dtype=float32, numpy=\narray([ 0.02436729, -0.02040631, -0.04043369,  0.03196613,  0.04834822,\n        0.02461036,  0.0206391 ,  0.10492345, -0.06469017,  0.03604339,\n       -0.02722654, -0.01371731,  0.06116929, -0.01725947,  0.03431245,\n       -0.09000932,  0.03107955, -0

### Export architecture to json file

In [104]:

json_string = global_model.to_json()

In [105]:
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 784], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_input"}}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "batch_input_shape": [null, 784], "units": 200, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Activation", "config": {"name": "activation", "trainable": true, "dtype": "float32", "activation": "relu"}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 200, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "Gl

### Export weights as h5 file

In [106]:
global_model.save_weights('weights.h5')

### Export weights as a list

In [110]:
import pickle

with open('weights.pkl', 'wb') as f:
    pickle.dump(average_weights, f)